# 方法一：使用类变量 + __new__（经典方式）

In [4]:
class Singleton:
    _instance = None  # 用于保存类的唯一实例

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            # 如果还没有创建实例，就调用父类创建实例
            cls._instance = super().__new__(cls)
        return cls._instance

# 测试
a = Singleton(1)
b = Singleton(2)
print(a is b)  # True，说明两个对象是同一个
print(a)  # 输出两个对象的内存地址，应该是相同的


True


In [8]:
class SingleTon:
    _instance = {} # 字典 _instance[cls]，支持多子类共用

    def __new__(cls, *args, **kwargs):
        if cls not in cls._instance:
            cls._instance[cls] = super().__new__(cls)
        return cls._instance[cls]


class MyClass(SingleTon):
    class_val = 44  # 类变量，所有实例共享
    # _initialized = False  # 防止 __init__ 多次执行（可选）

    def __init__(self, val):
    #     if not self._initialized:
              self.val = val
    #         MyClass._initialized = True

    def obj_fun(self):
        print(self.val, 'obj_fun')

    @staticmethod
    def static_fun():
        print('staticmethod')

    @classmethod
    def class_fun(cls):
        print(cls.class_val, 'classmethod')


a = MyClass(1)
b = MyClass(2)

print(a is b)            # True：单例验证
print(id(a), id(b))      # ID 相同
print(type(a))           # <class '__main__.MyClass'>
print(type(b))           # <class '__main__.MyClass'>

a.obj_fun()              # 输出 (1, 'obj_fun')，说明 val 没被第二次 __init__ 改写
b.obj_fun()              # 同样输出 (1, 'obj_fun')
MyClass.static_fun()     # staticmethod
MyClass.class_fun()      # 22 classmethod


True
4844888832 4844888832
<class '__main__.MyClass'>
<class '__main__.MyClass'>
2 obj_fun
2 obj_fun
staticmethod
44 classmethod


# 方法二：使用装饰器（函数式单例）

In [9]:
def singleton(cls):
    instances = {}  # 字典缓存类对应的实例

    def get_instance(*args, **kwargs):
        if cls not in instances:
            # 如果还没有这个类的实例，就创建并保存
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]

    return get_instance

@singleton
class MyClass:
    def __init__(self):
        print("初始化 MyClass")

# 测试
a = MyClass()
b = MyClass()
print(a is b)  # True


初始化 MyClass
True


In [12]:
from functools import wraps

# 单例装饰器
def single_ton(cls):
    _instance = {}  # 用字典缓存每个类对应的唯一实例

    @wraps(cls) # 保留原类的元信息（如 __name__, __doc__），调试友好
    def single(*args, **kwargs):
        if cls not in _instance:
            # 如果还没有创建过，就创建并保存
            _instance[cls] = cls(*args, **kwargs)
        return _instance[cls]

    return single  # 返回包装后的函数（代替原类）

# 使用装饰器实现单例
@single_ton
class SingleTon:
    val = 123

    def __init__(self, a):
        self.a = a

if __name__ == '__main__':
    s = SingleTon(1)
    t = SingleTon(2)

    print(s is t)           # True：单例验证
    print(s.a, t.a)         # 1 1：构造函数只执行一次，第二次不改变 a
    print(s.val, t.val)     # 123 123：类属性也一致


True
1 1
123 123


# 方法三：使用模块（Pythonic 最推荐）

In [ ]:
# singleton_module.py:
class MySingleton:
    def __init__(self):
        print("初始化 MySingleton")

singleton = MySingleton()  # 模块级别的对象


# main.py:
from singleton_module import singleton

a = singleton
b = singleton
print(a is b)  # True


In [ ]:
# use_module.py
class SingleTon(object):

    def __init__(self, val):
        self.val = val

single = SingleTon(2)


# test_module.py
from use_module import single

a = single
b = single
print (a.val, b.val)
print (a is b)
a.val = 233
print (a.val, b.val)


# 方法四：使用元类（进阶方式）

In [19]:
class SingletonMeta(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            # 调用父类的 __call__ 创建实例并缓存
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

class MyClass(metaclass=SingletonMeta):
    def __init__(self, val):
        self.val = val

class HerClass(metaclass=SingletonMeta):
    def __init__(self, val):
        self.val = val

# 测试
a = MyClass(3)
b = MyClass(4)
print(a is b)  # True

c = HerClass(5)
d = HerClass(6)
print(c is d)  # True
print(a is c)  # False，两个不同类的单例实例
print(a.val, b.val)  # 3 3，MyClass 的实例值
print(c.val, d.val)  # 5 5，HerClass 的实例值

True
True
False
3 3
5 5


# 方法五：线程安全（可用于面试加分）

In [21]:
import threading

class ThreadSafeSingleton:
    _instance = None # 用于保存唯一实例
    _lock = threading.Lock()  # 锁对象，确保线程安全

    def __new__(cls, *args, **kwargs):
        if cls._instance is None: # 第一次判断，提高性能（避免每次都加锁）
            with cls._lock: # 加锁，确保不会有多个线程同时进入。只有第一次创建时加锁
                if cls._instance is None: # 第二次判断，确保只有一个实例被创建
                    cls._instance = super().__new__(cls) # 创建实例
        return cls._instance

# 测试
a = ThreadSafeSingleton(1)
b = ThreadSafeSingleton(2)
print(a is b)  # True，说明两个对象是同一个
print(a)  # 输出两个对象的内存地址，应该是相同的


True


In [27]:
import threading
import time

class ThreadSafeSingleton:
    _instance = None
    _lock = threading.Lock()

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            with cls._lock:
                if cls._instance is None:
                    print(f"[{threading.current_thread().name}] 正在创建实例")
                    cls._instance = super().__new__(cls)
                    time.sleep(0.1)  # 模拟耗时操作
        return cls._instance

    def __init__(self):
        self.value = 42

# 测试函数：尝试获取单例
def create_instance():
    obj = ThreadSafeSingleton()
    print(f"[{threading.current_thread().name}] 实例ID：{id(obj)}")

if __name__ == "__main__":
    threads = [] # 创建一个空列表，用来保存后面创建的所有线程对象。这样可以统一管理线程，后面会用来 join() 等待它们结束
    for i in range(5):
        t = threading.Thread(target=create_instance, name=f"线程{i+1}") # 创建线程对象，指定目标函数和线程名称。当线程启动时，它就会执行这个函数
        threads.append(t) # 把创建好的线程对象 t 添加到列表 threads 中
        t.start() # 启动线程，开始执行目标函数。
        # “异步”执行。每个线程会并发调用 ThreadSafeSingleton()，也就是我们要测试的单例类

    for t in threads:
        t.join() # t.join() 会阻塞主线程（即 main 线程），直到线程 t 执行完毕
        #所以这一段的作用是：等待前面启动的所有线程执行完毕后再继续往下走；这样可以保证主程序不会提前结束，确保我们能看到所有线程的输出。


[线程1] 正在创建实例
[线程2] 实例ID：4844898016
[线程3] 实例ID：4844898016
[线程4] 实例ID：4844898016
[线程5] 实例ID：4844898016
[线程1] 实例ID：4844898016


#### 输出顺序解释
操作系统在运行多线程程序时，每个线程并不是按顺序依次执行，而是由操作系统调度器决定什么时候运行哪个线程、运行多长时间。

在你这个例子中：
线程1最先启动，进入 __new__() 方法。
它满足 if cls._instance is None。
打印了 [线程1] 正在创建实例
然后线程1被操作系统“暂停”了（中途挂起！），让线程2、3、4、5先运行完。
这些线程此时看到 _instance 已经不是 None（因为线程1已经创建了对象），所以直接拿到了对象。
打印了它们自己的 实例ID
最后线程1继续执行下半段（被唤醒），打印自己的 实例ID

✅ 为什么线程1先输出“正在创建实例”但最后才输出 实例ID？
因为：
print(f"[线程1] 正在创建实例") 是在锁内部，很早就执行了；
但接下来的 print(f"[线程1] 实例ID...") 是在锁外的 create_instance() 函数里；
操作系统可能在两次之间暂停了线程1的运行，把时间让给了其他线程；
所以其它线程“插队”打印了 实例ID，最后线程1才补上。